In [29]:
import os
import pandas as pd
import matplotlib.pyplot as plt

os.chdir('/Users/kelvin/Desktop/citadel datathon/')

acs =            pd.read_csv('data/acs_5yr_est_selected_economic_characteristics_2010-2022.csv')
commodities =    pd.read_csv('data/all_commodities.csv')
s_and_e =        pd.read_csv('data/all_stocks_and_etfs.csv')
e_consumption =  pd.read_csv('data/Energy Data - Consumption.csv')
e_emissions =    pd.read_csv('data/Energy Data - Emissions.csv')
e_goods =        pd.read_csv('data/Energy Data - Imports & Exports.csv')
e_prod =         pd.read_csv('data/Energy Data - Production.csv')
e_stocks =       pd.read_csv('data/Energy Data - Stocks.csv')
transportation = pd.read_csv('data/monthly_transportation_statistics.csv')
stock_desc =     pd.read_csv('data/stock_descriptions.csv')

In [30]:
# display(acs.head())
# display(commodities.head())
display(s_and_e.head())
# display(e_consumption.head())
# display(e_emissions.head())
# display(e_goods.head())
# display(e_prod.head())
# display(e_stocks.head())
# display(transportation.head())
# display(stock_desc.head())

,Date-Time,Open,High,Low,Close,Volume,Ticker_Symbol
0,2024-01-25,37.08,37.2481,35.9997,37.00,11492073,HAL
1,2024-01-24,35.68,36.9500,35.4838,36.83,14397273,HAL
2,2024-01-23,34.48,35.6900,33.9400,35.30,15096866,HAL
3,2024-01-22,33.88,34.6500,33.7750,34.43,11147116,HAL
4,2024-01-19,33.43,33.9800,33.3200,33.97,8908991,HAL


In [31]:
# ONEQ split adjustment
s_and_e['Date-Time'] = pd.to_datetime(s_and_e['Date-Time'])
split_day = pd.to_datetime('2021-04-08')
s_and_e.loc[(s_and_e['Date-Time'] >= split_day) &
            (s_and_e['Ticker_Symbol'] == 'ONEQ'),
            ['Open', 'High', 'Low', 'Close']] *= 10

In [32]:
pd.set_option('display.max_rows', None)

In [33]:
from IPython.display import display, HTML

display(HTML("""
<style>
.output_scroll {
    height: auto !important;
    max-height: 100px; /* Adjust this value as needed */
}
</style>
"""))

In [34]:
def plot_moving_average(result, ticker_symbol):
    # Set the 'Date-Time' column as the index
    result.set_index('Date-Time', inplace=True)
    
    # Plotting the 'Close' price and the 'Moving_Avg'
    plt.figure(figsize=(14,7))  # Set the figure size as needed
    plt.plot(result['Close'], label='Close Price')
    plt.plot(result['Moving_Avg'], label='Moving Average', linewidth=2)
    
    # Title and labels
    plt.title('Moving Average of Close Prices Over Time for ' + ticker_symbol)
    plt.xlabel('Date')
    plt.ylabel('Price')
    
    # Legend
    plt.legend()
    
    # Show the plot
    plt.show()


In [35]:
def compute_moving_average(df, ticker_symbol, window_size=60):
    """
    Compute the moving average for the 'Close' prices of a given ticker symbol.

    Parameters:
    df (pandas.DataFrame): The DataFrame containing stock data.
    ticker_symbol (str): The ticker symbol to filter the DataFrame.
    window_size (int): The size of the window for the moving average.

    Returns:
    pandas.DataFrame: A DataFrame with 'Date-Time', 'Close', and 'Moving_Avg' for the ticker symbol.
    """
    # Create a copy to avoid the SettingWithCopyWarning
    ticker_df = df[df['Ticker_Symbol'] == ticker_symbol].copy()
    
    # Sort the dataframe based on date to ensure correct moving average calculation
    ticker_df.sort_values('Date-Time', inplace=True)
    
    # Compute moving average of 'Close' prices
    ticker_df['Moving_Avg'] = ticker_df['Close'].rolling(window=window_size).mean()
    
    ret = ticker_df[['Date-Time', 'Close', 'Moving_Avg']]
    plot_moving_average(ticker_df[['Date-Time', 'Close', 'Moving_Avg']], ticker_symbol)
    
    return ret


In [38]:
# for ticker in s_and_e['Ticker_Symbol'].unique():
#     compute_moving_average(s_and_e, ticker)

In [39]:
pivoted_s_e = s_and_e.pivot(index='Date-Time', columns='Ticker_Symbol', values='Close')

In [44]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

daily_returns = pivoted_s_e.pct_change() * 1000
covariance_matrix = daily_returns.cov()

pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')

print(covariance_matrix)

Ticker_Symbol         BKR          BP          BPT         CEO         CHK  \
Ticker_Symbol                                                                
BKR            712.229034  368.979882   454.028176  204.514061  278.569381   
BP             368.979882  364.124293   242.054049  221.022496  242.905334   
BPT            454.028176  242.054049  1071.851599  213.206602  512.903751   
CEO            204.514061  221.022496   213.206602  528.455908    0.687782   
CHK            278.569381  242.905334   512.903751    0.687782  592.162704   
COP            477.193881  288.589343   302.439975  263.715136  315.659693   
CVX            385.289181  246.314895   225.283095  218.616015  222.948007   
DIA            189.057452  128.740222    81.524049  142.329744   89.924803   
DVN            627.474100  343.631743   399.294776  321.456661  411.692003   
E              320.009297  257.931050   203.746716  232.510750  189.645787   
EOG            502.022870  306.365477   344.037988  295.430133  

/var/folders/_0/4yp2qnxj3hn3g3w46xt2132w0000gn/T/ipykernel_44423/2509954787.py:4: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  daily_returns = pivoted_s_e.pct_change() * 1000
